In [ ]:
import timeit

import numpy as np
import ChebTools
import matplotlib.pyplot as plt
import pandas

In [ ]:
# https://www.chebfun.org/examples/approx2/PrettyFunctions.html
f = lambda x, y: np.cos(10*(x*x+y))*np.sin(10*(x+y*y))

In [ ]:
# Time construction of expansions of given NxN size 
o = []
for N in range(1,20):
    tic = timeit.default_timer()
    b = ChebTools.ChebyshevExpansion2DBounds()
    ce = ChebTools.generate_Chebyshev_expansion2D(f, [N, N], b)
    toc = timeit.default_timer()
    elap = toc-tic
    o.append({'N': N, 'elap': elap})
df = pandas.DataFrame(o)
plt.plot(df.N, df.elap, 'o-')
plt.yscale('log')
plt.xlabel('N')
plt.ylabel('time')
plt.show()

In [ ]:
b = ChebTools.ChebyshevExpansion2DBounds()
for w in [1, 1/2, 1/4, 1/8, 1/16, 1/32, 1/64]:

    b.xmin = -w
    b.xmax = w
    b.ymin = -w
    b.ymax = w

    ce = ChebTools.generate_Chebyshev_expansion2D(f, [8, 8], b)

    o = []
    for x_ in np.linspace(-w, w, 100):
        for y_ in np.linspace(-w, w, 100):
            val = f(x_, y_)
            err = ce.eval_Clenshaw(x_, y_) - val
            o.append({'x': x_, 'y': y_, 'val': val, 'err': err})
    df = pandas.DataFrame(o)

    print(w)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

    sc = ax1.scatter(df.x, df.y, c=df.val, marker='s', s=4)
    plt.sca(ax1)
    cb = plt.colorbar(sc)
    cb.set_label('val')

    sc = ax2.scatter(df.x, df.y, c=df.err, s=1)
    plt.sca(ax2)
    cb = plt.colorbar(sc)
    cb.set_label('error')

    plt.tight_layout(pad=0.2)
    plt.show()

In [ ]:
b = ChebTools.ChebyshevExpansion2DBounds()
for xmin in [-1, -0.5, 0, 0.5, 0.75, 0.9, 0.95]:

    b.xmin = xmin
    b.xmax = 1
    b.ymin = xmin
    b.ymax = 1

    ce = ChebTools.generate_Chebyshev_expansion2D(f, [8, 8], b)

    o = []
    for x_ in np.linspace(b.xmin, b.xmax, 100):
        for y_ in np.linspace(b.xmin, b.ymax, 100):
            val = f(x_, y_)
            err = ce.eval_Clenshaw(x_, y_) - val
            o.append({'x': x_, 'y': y_, 'val': val, 'err': err})
    df = pandas.DataFrame(o)

    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

    sc = ax1.scatter(df.x, df.y, c=df.val, marker='s', s=4)
    plt.sca(ax1)
    cb = plt.colorbar(sc)
    cb.set_label('val')

    sc = ax2.scatter(df.x, df.y, c=df.err, s=1)
    plt.sca(ax2)
    cb = plt.colorbar(sc)
    cb.set_label('error')

    plt.tight_layout(pad=0.2)
    plt.show()

In [ ]:
def tiled_domain(f, *, ob, Nx, Ny):
    """
    ob: overall bounds
    """
    xs = np.linspace(ob.xmin, ob.xmax, Nx+1)
    ys = np.linspace(ob.ymin, ob.ymax, Ny+1)
    ohs = []
    for i in range(len(xs)-1):
        for j in range(len(ys)-1):
            
            # One domain
            b = ChebTools.ChebyshevExpansion2DBounds()
            b.xmin = xs[i]
            b.xmax = xs[i+1]
            b.ymin = ys[j]
            b.ymax = ys[j+1]
            ce = ChebTools.generate_Chebyshev_expansion2D(f, [10, 10], b)
            o = []
            for x_ in np.linspace(b.xmin, b.xmax, 20):
                for y_ in np.linspace(b.ymin, b.ymax, 20):
                    val = f(x_, y_)
                    err = ce.eval_Clenshaw(x_, y_) - val
                    o.append({'x': x_, 'y': y_, 'val': val, 'err': err})
                    
            # Store the results
            ohs += o
            
    # Join all the results together
    df = pandas.DataFrame(ohs)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

    sc = ax1.scatter(df.x, df.y, c=df.val, marker='s', s=4)
    plt.sca(ax1)
    cb = plt.colorbar(sc)
    cb.set_label('val')

    sc = ax2.scatter(df.x, df.y, c=df.err, s=1)
    plt.sca(ax2)
    cb = plt.colorbar(sc)
    cb.set_label('error')

    for i in range(len(xs)):
        for j in range(len(ys)):
            plt.plot(xs[i], ys[j], 'ro')
    
    byt = Nx*Ny*8*11*11/1024/1024
    plt.suptitle(f'{Nx} x {Ny}; {byt} MB')

    plt.tight_layout(pad=0.2)
    plt.show()
    

In [ ]:
for p in range(1, 7, 1):
    N = 2**p
    tiled_domain(f, ob=ChebTools.ChebyshevExpansion2DBounds(), Nx=N, Ny=N)

In [ ]:
for p in range(1, 7, 1):
    N = 2**p
    tiled_domain(lambda x,y: np.cos(10*(x))*np.sin(10*(y)), ob=ChebTools.ChebyshevExpansion2DBounds(), Nx=N, Ny=N)